<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> **Finanzas Cuantitativas | Otoño 2021**. 

**Nombres:** 
- Juan Pablo Ruíz.
- Rubén Hernández.
- Bryan Juárez.

**Fecha:** 27 de mayo del 2021.
    
<img style="float: right; margin: 30px 15px 15px 15px;" src="https://www.monash.edu/__data/assets/image/0004/1017517/iStock-611890326-OK3.jpg" width="300" height="500" />

**Expedientes** : 
- 721093.
- 717710.
- 722176.

**Profesor:** José Mario Zárate.  

## Tarea 1.

### Ejercicio 1.1

Use the Csv files that contains 2000 records of the sales of similar products in the (each columns represents a year).

Develop a function in python where you can evaluate a project usingIRR and NPV.

The inputs is the flows by year (csv file), the output is a data frame containing NPV and IRR. 

Add lines to the code to answer the 4 question of the activity:
- 1°- What the probability that the IRR is over the Risk free rate?
- 2°- What the probability that the IRR is over the 35%?
- 3°- What the probability that the project value is over $\$2\text{M}$?
- 4°- What’s the probability that the IRR is between $10\%$ and $20\%$?

In [ ]:
# Solution.
.
.
.

### Ejercicio 1.2

Select one of the options from slide 4 and propose adding theuncertainty factor in one of the variables.
- 1°- Explain clearly your conditions.
- 2°- Draw the decision tree and expected value.
- 3°- Explain under what conditions your decisions changes.

In [ ]:
# Solution.
.
.
.